In [16]:
from bs4 import BeautifulSoup
import requests, json
from splinter import Browser
import datetime
from config import gkey

In [17]:
executable_path = {'executable_path': 'static/webdriver/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [18]:
#https://www.fandango.com/44107_movietimes?mode=general&q=44107&date=2019-01-20
zipcode = "44129"
#movie_date = "2019-01-23"
movie_date = datetime.datetime.today().strftime('%Y-%m-%d')
url = ("https://www.fandango.com/{0}_movietimes?mode=general&q={0}&date={1}").format(zipcode, movie_date)
print(url)

https://www.fandango.com/44129_movietimes?mode=general&q=44129&date=2019-01-22


In [19]:
browser.visit(url)
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'lxml')

In [20]:
#page_response = requests.get(url, timeout=5)
# here, we fetch the content from the url, using the requests library
#soup = BeautifulSoup(page_response.content, "lxml")
#we use the html parser to parse the url content and store it in a variable.

In [21]:
showtimes = soup.find('div',class_='fd-showtimes')

In [22]:
theaters = showtimes.find_all('li', class_='fd-theater')

In [23]:
#create empty dictionary
theater_data = []
#Find theater_names, theater locations, movies, and showtimes.
theater_names = []
theater_locations = []
theaters_movies = []
for i in range(0,len(theaters)):
    theater_name = theaters[i].find('a',class_='light').text
    theater_names.append(theater_name)
    theaterlocation = theaters[i].find('div',class_='fd-theater__address-wrap').text
    theater_location = ' '.join(theaterlocation.rstrip().split())
    theater_locations.append(theater_location)
    movies = theaters[i].find_all('li',class_='fd-movie')
    theater_movies = []
    movies_titles = []
    movies_showtimes = []
    movies_posters = []
    target = theater_location
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target, gkey)
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
    lat = geo_data["results"][0]["geometry"]["location"]["lat"]
    lng = geo_data["results"][0]["geometry"]["location"]["lng"]
    for k in range(0,len(movies)):
        if movies[k].find('a',class_='dark'):
            movie_title = movies[k].find('a',class_='dark').text
        else:
            movie_title = movies[k].find('a',class_='dark')
        movies_titles.append(movie_title)
        if movies[k].find('img'):
            movie_poster = movies[k].find('img')['src']
        else:
            movie_poster = movies[k].find('img')
        movies_posters.append(movie_poster)
        movie_showtimes = []
        if movies[k].find_all('a',class_='showtime-btn--available'):
            numshows=movies[k].find_all('a',class_='showtime-btn--available')
        else:
            numshows=movies[k].find_all('span',class_='showtime-btn--non-ticketing')
        for j in range(0,len(numshows)):
            showtime = numshows[j].text
            movie_showtimes.append(showtime)
        movies_showtimes.append(movie_showtimes)
        theater_movies.append({'Title':movies_titles[k],'Showtimes':movies_showtimes[k],'Poster_URL':movies_posters[k]})
    theaters_movies.append(theater_movies)
    theater_data.append({'Name':theater_names[i], 'Address':theater_locations[i],'lat':lat,'lng':lng,'Movies':theaters_movies[i]})

In [24]:
numshows=movies[k].find_all('span',class_='showtime-btn--non-ticketing')
numshows[0].text

'1:20p'

In [25]:
theater_data[0]['Movies'][5]['Title']

'Bumblebee'

In [27]:
print(json.dumps(theater_data, indent=4, sort_keys=True))

[
    {
        "Address": "4788 Ridge Road, Brooklyn, OH 44144",
        "Movies": [
            {
                "Poster_URL": "//images.fandango.com/ImageRenderer/200/0/redesign/static/img/default_poster.png/0/images/MasterRepository/fandango/213141/Glass.jpg",
                "Showtimes": [
                    "9:30p",
                    "10:00p"
                ],
                "Title": "Glass (2019)"
            },
            {
                "Poster_URL": "//images.fandango.com/ImageRenderer/200/0/redesign/static/img/default_poster.png/0/images/MasterRepository/fandango/213339/IfBealeStreetCouldTalk2018.jpg",
                "Showtimes": [
                    "9:55p"
                ],
                "Title": "If Beale Street Could Talk"
            },
            {
                "Poster_URL": "//images.fandango.com/ImageRenderer/200/0/redesign/static/img/default_poster.png/0/images/MasterRepository/fandango/214774/TheUpside2018.jpg",
                "Showtimes": [
    

In [33]:
    with open('static/result' + zipcode + '.json', 'w') as fp:
        json.dump(theater_data, fp)